In [1]:
import pandas as pd
from datetime import datetime as dt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
! ls

clean_requetes311_2014-2016.csv  clean_requetes311_2022-present.csv
clean_requetes311_2016-2018.csv  preliminary_eda_IA.ipynb
clean_requetes311_2019-2021.csv


In [9]:
data_2014_2016 = pd.read_csv('requetes311_2014-2016.csv', low_memory=False)
data_2016_2018 = pd.read_csv('requetes311_2016-2018.csv', low_memory=False)
data_2019_2021 = pd.read_csv('requetes311_2019-2021.csv', low_memory=False)
data_2022_present = pd.read_csv('requetes311_2022-present.csv', low_memory=False)

In [17]:
NAMES = ['2014-2016', '2016-2018', '2019-2021', '2022-present']
DATASETS = [data_2014_2016, data_2016_2018, data_2019_2021, data_2022_present]
DATA_DICT = dict(zip(NAMES,DATASETS))

In [19]:
DATA_DICT.keys()

dict_keys(['requetes311_2014-2016.csv', 'requetes311_2016-2018.csv', 'requetes311_2019-2021.csv', 'requetes311_2022-present.csv'])

# CLEAN: remove all rows without geo-spacial data
SAVE output csv

In [22]:
for key in DATA_DICT.keys():
    print(f'processing {key}...')
    data = DATA_DICT[key]
    data.dropna(subset=['LOC_LONG'], inplace=True)
    
    data = data[data['NATURE'] != 'Commentaire']
    
    data_clean = data.copy()
    
    data_clean.to_csv(f"clean_{key}")
    print(f'clean_{key} complete')

processing requetes311_2014-2016.csv...
clean_requetes311_2014-2016.csv complete
processing requetes311_2016-2018.csv...
clean_requetes311_2016-2018.csv complete
processing requetes311_2019-2021.csv...
clean_requetes311_2019-2021.csv complete
processing requetes311_2022-present.csv...
clean_requetes311_2022-present.csv complete


# LOAD cleaned data

In [3]:
data_2014_2016 = pd.read_csv('clean_requetes311_2014-2016.csv', low_memory=False)
data_2016_2018 = pd.read_csv('clean_requetes311_2016-2018.csv', low_memory=False)
data_2019_2021 = pd.read_csv('clean_requetes311_2019-2021.csv', low_memory=False)
data_2022_present = pd.read_csv('clean_requetes311_2022-present.csv', low_memory=False)

In [4]:
NAMES = ['2014-2016', '2016-2018', '2019-2021', '2022-present']
DATASETS = [data_2014_2016, data_2016_2018, data_2019_2021, data_2022_present]
DATA_DICT = dict(zip(NAMES,DATASETS))

# Count number of incidents per period

In [5]:
for key in DATA_DICT:
    print(f' {key}:')
    data = DATA_DICT[key]
    data.value_counts('NATURE')

 2014-2016:


NATURE
Requete    1060340
Plainte      14348
Name: count, dtype: int64

 2016-2018:


NATURE
Requete    1172488
Plainte      15374
Name: count, dtype: int64

 2019-2021:


NATURE
Requete    1014315
Plainte      18486
Name: count, dtype: int64

 2022-present:


NATURE
Requete    1013817
Plainte      17308
Name: count, dtype: int64

In [28]:
print(f"there are {14348+15374+18486+17308} complaints")

65516

# preliminary analysis

In [6]:
data = data_2022_present

In [7]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031125 entries, 0 to 1031124
Data columns (total 30 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1031125 non-null  int64  
 1   ID_UNIQUE                1031125 non-null  object 
 2   NATURE                   1031125 non-null  object 
 3   ACTI_NOM                 1031125 non-null  object 
 4   TYPE_LIEU_INTERV         1031125 non-null  object 
 5   RUE                      884315 non-null   object 
 6   RUE_INTERSECTION1        158788 non-null   object 
 7   RUE_INTERSECTION2        158686 non-null   object 
 8   LOC_ERREUR_GDT           1031125 non-null  float64
 9   ARRONDISSEMENT           1031125 non-null  object 
 10  ARRONDISSEMENT_GEO       1031083 non-null  object 
 11  LIN_CODE_POSTAL          802661 non-null   object 
 12  DDS_DATE_CREATION        1031125 non-null  object 
 13  PROVENANCE_ORIGINALE     1029861 non-null 

In [8]:
data.value_counts('NATURE')

NATURE
Requete    1013817
Plainte      17308
Name: count, dtype: int64

In [9]:
data.value_counts(['DERNIER_STATUT'])

DERNIER_STATUT           
Terminée                     963236
Refusée                       28066
Prise en charge               22387
Transmise pour traitement      7040
Annulée                        6272
Acceptée                       3475
Supprimée                       515
Réactivée                       112
Urgente                          22
Name: count, dtype: int64

In [10]:
DATETIME_COLS = ['DDS_DATE_CREATION','DATE_DERNIER_STATUT']
data[DATETIME_COLS[0]] = pd.to_datetime(data[DATETIME_COLS[0]])
data[DATETIME_COLS[1]] = pd.to_datetime(data[DATETIME_COLS[1]])

In [14]:
pd.set_option('display.max_rows', 100)
data.value_counts('ACTI_NOM').head(100)

ACTI_NOM
Dépôt illégal - Déchets                             61164
Collecte de déchets                                 42612
Bac roulant                                         38072
Nettoyage du domaine public                         33014
Collecte des matières recyclables                   29909
Éclairage existant - Entretien                      28713
Opération déneigement                               28313
Nid-de-poule                                        26298
Feux de circulation - Entretien                     22215
Élagage                                             20838
Signalisation - Circulation - Études                20804
Collecte de résidus alimentaires                    20400
Fermeture d'entrée d'eau                            19368
Branche tombée                                      16305
Occupation du domaine public                        15538
Pavage - Réparation                                 15035
Puisard/TH-Première intervention                    14362
Colle

In [15]:
data.value_counts(['DERNIER_STATUT'])

DERNIER_STATUT           
Terminée                     963236
Refusée                       28066
Prise en charge               22387
Transmise pour traitement      7040
Annulée                        6272
Acceptée                       3475
Supprimée                       515
Réactivée                       112
Urgente                          22
Name: count, dtype: int64

In [16]:
(data[DATETIME_COLS[1]] - data[DATETIME_COLS[0]]).sort_values().describe()

count                       1031125
mean     26 days 00:23:12.030365862
std      69 days 07:19:55.236363579
min                 0 days 00:00:00
25%                 0 days 22:11:17
50%                 4 days 05:51:03
75%                17 days 06:43:21
max              1077 days 20:16:59
dtype: object

In [18]:
data['TERMINATION_TIME'] = (data[DATETIME_COLS[1]] - data[DATETIME_COLS[0]])

In [19]:
data[['ARRONDISSEMENT','TERMINATION_TIME']]

,ARRONDISSEMENT,TERMINATION_TIME
0,Ahuntsic - Cartierville,2 days 03:32:50
1,Outremont,1 days 03:03:46
2,Le Plateau-Mont-Royal,5 days 05:21:48
3,Saint-Léonard,5 days 14:53:38
4,Ahuntsic - Cartierville,2 days 23:09:14
...,...,...
1031120,Le Plateau-Mont-Royal,0 days 04:42:47
1031121,Côte-des-Neiges - Notre-Dame-de-Grâce,0 days 07:38:27
1031122,Rivière-des-Prairies - Pointe-aux-Trembles,0 days 03:18:29
1031123,Rosemont - La Petite-Patrie,0 days 10:28:24


In [21]:
data.loc[data['ARRONDISSEMENT']=='Ahuntsic - Cartierville','TERMINATION_TIME'].describe()

count                         86261
mean     32 days 18:15:45.538876201
std      76 days 10:11:57.299294605
min                 0 days 00:00:00
25%                 1 days 01:47:39
50%                 4 days 20:49:40
75%                31 days 23:32:08
max               998 days 02:19:42
Name: TERMINATION_TIME, dtype: object

In [22]:
data.loc[data['ARRONDISSEMENT']=='Le Plateau-Mont-Royal','TERMINATION_TIME'].describe()

count                         80695
mean     22 days 23:23:01.739153603
std      55 days 07:36:20.107497937
min                 0 days 00:00:00
25%                 1 days 01:33:55
50%                 4 days 02:31:53
75%         17 days 08:55:40.500000
max               984 days 04:29:46
Name: TERMINATION_TIME, dtype: object

In [26]:
HAZARDS = [
    "Feux de circulation - Entretien",         #'Traffic lights - Maintenance',
    "Branche tombée",                          # 'Fallen branch',
    "Débris sur la voie publique",             # 'Debris on the public highway',
    "Fuite d'eau",                             # 'Water leak',
    "Trottoir glissant ",                      # 'Slippery sidewalk',
    "Chaussée glissante",                      # 'Slippery road',
    "Fermeture d'une conduite d'eau - Urgence",# 'Closure of a water pipe - Emergency'
]

In [27]:
data.value_counts('ACTI_NOM')

ACTI_NOM
Dépôt illégal - Déchets                                  61164
Collecte de déchets                                      42612
Bac roulant                                              38072
Nettoyage du domaine public                              33014
Collecte des matières recyclables                        29909
                                                         ...  
*Refuge - Propreté                                           1
Constat d'infraction - Neige                                 1
*Puisard/TH-Refoulement/Débordement/Blocage/Clés             1
*Puisard/TH-Couvercle Abs./Brisé/Déplacé/Affaissement        1
*ÉLU - Direction d'arrondissement                            1
Name: count, Length: 868, dtype: int64

In [28]:
data.loc[data['ACTI_NOM'].isin(HAZARDS),'TERMINATION_TIME'].describe()

count                         64180
mean     12 days 16:47:03.336771579
std      43 days 21:50:49.828612146
min                 0 days 00:00:00
25%                 0 days 02:42:53
50%                 1 days 02:07:30
75%          7 days 05:19:45.750000
max               993 days 15:48:50
Name: TERMINATION_TIME, dtype: object